In [1]:
# %%bash
# pip install lightfm

In [2]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
from sklearn import model_selection
from sklearn.model_selection import KFold
from scipy.sparse import coo_matrix
from time import time
import matplotlib.pyplot as plt
import csv

import numpy as np
import pandas as pd

# from google.colab import files

/Users/joshfeldman/Desktop/Projects/personalization_project/venv/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


## Util

In [3]:
def downloadCSVFile(listOfDicts, fileName):
  resultDF = pd.DataFrame(listOfDicts)
  resultDF.to_csv(fileName)
#   files.download(fileName)

##Get Data

In [4]:
# %%bash
# mkdir -p 'results'

# rm ratings.csv

In [ ]:
# # Upload ratings.csv

# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

### Make Train, Validate, Test

In [ ]:
# ratingsDataPath = 'data-medium/ratings_8326033.csv'
ratingsDataPath = 'data-medium/ratings_1387672.csv'
# ratingsDataPath = 'ml-latest-small/ratings.csv'

ratingsDF = pd.read_csv(ratingsDataPath)
ratingsDF = ratingsDF[['userId', 'movieId', 'rating']]

# Alter ratings to work better for our model
ratingsDF['rating'] = ratingsDF['rating'].apply(lambda x: x - 2.9)

ratingsDF['userId'] = ratingsDF['userId'].astype('int32')
ratingsDF['movieId'] = ratingsDF['movieId'].astype('int32')

userIds = ratingsDF['userId'].unique()
movieIds = ratingsDF['movieId'].unique()

movieIdToIndex = {}
for index, movieId in enumerate(movieIds):
  movieIdToIndex[movieId] = index

userIdToIndex = {}
for index, userId in enumerate(userIds):
  userIdToIndex[userId] = index

ratingsTrainDF, ratingsTestDF = model_selection.train_test_split(ratingsDF, test_size=0.2, random_state=324)

# We want to do k-fold validation, so we create k differen't versions of the data
ratingsTrainArray = ratingsTrainDF.values

k = 3
trainValidateArrays = []
kFold = KFold(n_splits=k, random_state=299, shuffle=True)
for train_index, test_index in kFold.split(ratingsTrainArray):
  ratingsTrainGridSearch, ratingsValidateGridSearch = ratingsTrainArray[train_index], ratingsTrainArray[test_index]
  trainValidateArrays.append((ratingsTrainGridSearch, ratingsValidateGridSearch))
# ratingsTrainGridSearchDF, ratingsValidateGridSearchDF = model_selection.train_test_split(ratingsTrainDF, test_size=0.2, random_state=773)

print('We have {} users'.format(len(userIds)))
print('We have {} movies'.format(len(movieIds)))
print('We have {} train ratings'.format(len(ratingsTrainDF)))
print('We have {} test ratings'.format(len(ratingsTestDF)))
print('We have {} grid search train ratings for each of {} folds'.format(len(trainValidateArrays[0][0]), k))
print('We have {} grid search validate ratings for each of {} folds'.format(len(trainValidateArrays[0][1]), k))

We have 203750 users
We have 24174 movies
We have 1110137 train ratings
We have 277535 test ratings
We have 740091 grid search train ratings for each of 3 folds
We have 370046 grid search validate ratings for each of 3 folds


In [ ]:
ratingsTrainDF.head()

,userId,movieId,rating
1385920,258790,594,-0.4
18532,211231,2797,-0.4
784929,198557,356,1.1
812525,266597,166486,0.1
608720,191893,434,1.1


In [ ]:
trainValidateArrays[0][0][1:10]

array([[ 2.11231e+05,  2.79700e+03, -4.00000e-01],
       [ 1.98557e+05,  3.56000e+02,  1.10000e+00],
       [ 1.91893e+05,  4.34000e+02,  1.10000e+00],
       [ 4.16520e+04,  1.56900e+03, -1.90000e+00],
       [ 6.05000e+02,  1.25800e+03, -1.90000e+00],
       [ 2.42800e+05,  5.39100e+03,  1.10000e+00],
       [ 1.78749e+05,  5.52470e+04,  6.00000e-01],
       [ 7.76090e+04,  6.11500e+03, -9.00000e-01],
       [ 1.79327e+05,  4.69670e+04,  6.00000e-01]])

### Convert to Matrix

In [ ]:
def ratingsDFToUserMovieMatrix(ratingsDF, userIdToIndex, movieIdToIndex): 
  return ratingsArrayToUserMovieMatrix(ratingsDF.values, userIdToIndex, movieIdToIndex)

def ratingsArrayToUserMovieMatrix(ratings, userIdToIndex, movieIdToIndex):
  numUsers = len(userIdToIndex)
  numMovies = len(movieIdToIndex)
  
  userMovieMatrix = np.zeros((numUsers, numMovies))
  
  for userId, movieId, rating in ratings:
    userId = int(userId)
    movieId = int(movieId)
    rating = float(rating)
    
    userIndex = userIdToIndex[userId]
    movieIndex = movieIdToIndex[movieId]
    userMovieMatrix[userIndex, movieIndex] = rating
    
  
  return coo_matrix(userMovieMatrix)


XTrainValidateFolds = []
for ratingsTrain, ratingsValidate in trainValidateArrays:
  XTrain = ratingsArrayToUserMovieMatrix(ratingsTrain, userIdToIndex, movieIdToIndex)
  XValidate = ratingsArrayToUserMovieMatrix(ratingsValidate, userIdToIndex, movieIdToIndex)
  XTrainValidateFolds.append((XTrain, XValidate))


print('Shape of each X Train: {}'.format(XTrainValidateFolds[0][0].shape))
print('Shape of each X Validate: {}'.format(XTrainValidateFolds[0][1].shape))

XTrain = ratingsArrayToUserMovieMatrix(ratingsTrainArray, userIdToIndex, movieIdToIndex)
XTest = ratingsDFToUserMovieMatrix(ratingsTestDF, userIdToIndex, movieIdToIndex)

print('')
print('Shape of Total X Train: {}'.format(XTrain.shape))
print('Shape of X Test: {}'.format(XTest.shape))

Shape of X Train: (203750, 24174)
Shape of X Test: (203750, 24174)

Shape of each X Train: (203750, 24174)
Shape of each X Validate: (203750, 24174)


# Find Baseline Score

In [ ]:
def calculateAUC(userStats):
  tp = userStats['tp']
  fp = userStats['fp']
  tn = userStats['tn']
  fn = userStats['fn']
  if tp + fn == 0 or tn + fp == 0:
    return -5
  tpr = tp / (tp + fn)
  fpr = fp / (tn + fp)
  auc = (tpr - fpr + 1) / 2
  return auc

In [ ]:
averageMovieRatings = np.asarray(np.mean(XTrain, axis=0))[0]

numUsers = XTest.shape[0]
numMovies = XTest.shape[1]


startingStats = {
  'tp': 0,
  'fp': 0,
  'tn': 0,
  'fn': 0
}

currentUserIndex = -1
currentUserStats = {**startingStats}
aucPerUser = np.zeros(numUsers)

for userIndex,movieIndex,ratingActual in zip(XTest.row, XTest.col, XTest.data):
  if userIndex != currentUserIndex:
    aucPerUser[currentUserIndex] = calculateAUC(currentUserStats)
    currentUserStats = {**startingStats}
    currentUserIndex = userIndex
  
  if ratingActual != 0:
    ratingPredicted = averageMovieRatings[movieIndex]
    if ratingActual > 0:
      if ratingPredicted > 0:
        currentUserStats['tp'] += 1
      else:
        currentUserStats['fn'] += 1
    else:
      if ratingPredicted > 0:
        currentUserStats['fp'] += 1
      else:
        currentUserStats['tn'] += 1

aucPerUser = [auc for auc in aucPerUser if auc != -5]
baselineAUC = np.mean(aucPerUser)
print('Our baseline AUC is {}'.format(baselineAUC))

Our baseline AUC is 0.10802340452518355


# Run Grid Search

In [ ]:
def runModel(params, XTrain, XTest, numEpochs):
  model = LightFM(**params)
  
  startTime = time()
  model.fit(XTrain, epochs=numEpochs)
  
  totalTime = time() - startTime

  train_auc = auc_score(model, XTrain).mean()
  test_auc = auc_score(model, XTest).mean()
  
  return train_auc, test_auc, totalTime

In [ ]:
gridSearchParams = {
    'learning_rate': [0.05, 0.1, 0.15],
    'loss': ['bpr', 'warp', 'warp-kos'],
    'learning_schedule': ['adagrad', 'adadelta'],
    'no_components': [5, 8, 10, 14],
    'item_alpha': [0.0, 0.0001, 0.001, 0.01],
    'user_alpha': [0.0, 0.0001, 0.001, 0.01],
}

def gridSearch(gridSearchParams, XTrainValidateFolds):
  numEpochs = 4
  
  gridSearchResults = {}
  for gridSearchParam, paramValues in gridSearchParams.items():
    print('Running for parameter: {}'.format(gridSearchParam))
    gridSearchResults[gridSearchParam] = []
    for paramValue in paramValues:
      params = {
          gridSearchParam: paramValue
      }
      
      trainAUCs = []
      testAUCs = []
      times = []
      for XTrain, XValidate in XTrainValidateFolds:
        trainAUC, testAUC, time = runModel(params, XTrain, XValidate, numEpochs)
        trainAUCs.append(trainAUC)
        testAUCs.append(testAUC)
        times.append(time)
      
      
      result = {
          'value': paramValue,
          'train_auc': np.mean(trainAUCs),
          'test_auc': np.mean(testAUC),
          'time': np.mean(time),
      }
      gridSearchResults[gridSearchParam].append(result)
        
  return gridSearchResults

In [ ]:
startTime = time()

gridSearchResults = gridSearch(gridSearchParams, XTrainValidateFolds)
print('')
print(gridSearchResults)
# downloadCSVFile(gridSearchResults, 'gridSearchResults.csv')

totalTime = time() - startTime
if totalTime > 60:
    print('Total time: {:0.2f} minutes'.format(totalTime / 60))
else:
    print('Total time: {:0.2f} seconds'.format(totalTime))

Running for parameter: learning_rate


In [ ]:
def extractBestParams(gridSearchResults):
  bestParams = {}
  
  for parameter, parameterResults in gridSearchResults.items():
    bestIndex = 0
    bestAUC = parameterResults[0]['test_auc']

    for index, stats in enumerate(parameterResults):
      if stats['test_auc'] > bestAUC:
        bestAUC = stats['test_auc']
        bestIndex = index
    
    bestStats = parameterResults[bestIndex]
    bestParams[parameter] = bestStats['value']
  
    print('Best AUC for {} is: {}'.format(parameter, bestAUC))
  return bestParams

bestParams = extractBestParams(gridSearchResults)
bestParams

In [ ]:
def plotGridSearchResults(gridSearchResults):
  for param, paramResults in gridSearchResults.items():
    gridSearchDF = pd.DataFrame(paramResults)
   
    barWidth = 0.3
    barSpace = 0.16
    _X = np.arange(len(gridSearchDF['value']))
    plt.bar(_X - barSpace, gridSearchDF['train_auc'], barWidth, color='blue')
    plt.bar(_X + barSpace, gridSearchDF['test_auc'], barWidth, color='green')
    plt.xticks(_X, gridSearchDF['value'])
    plt.title('AUC -- {}'.format(param))
    plt.ylabel('AUC')
    plt.xlabel(param)
    plt.legend(['train', 'test'], loc='best')
    plt.show()

    plt.bar(_X, gridSearchDF['time'], barWidth, color='red')
    plt.xticks(_X, gridSearchDF['value'])
    plt.title('Time -- {}'.format('time'))
    plt.ylabel('Time')
    plt.xlabel(param)
    plt.legend(['train'], loc='best')
    plt.show()
  
plotGridSearchResults(gridSearchResults)

## Find Best Number of Epochs

In [ ]:
def runModelOverEpochs(params, XTrain, XTest, maxEpochs): 
  model = LightFM(**params)
  results = []
  
  runningTotalTime = 0
  for epoch in range(1, maxEpochs + 1)
    startTime = time()
    model.fit_partial(XTrain)
    totalTime = time() - startTime
    runningTotalTime += totalTime
    train_auc = auc_score(model, XTrain).mean()
    test_auc = auc_score(model, XTest).mean()
    results.append((train_auc, test_auc, runningTotalTime, epoch))
  return results

In [ ]:
def epochSearch(XTrain, XValidate, maxEpochs):
  
  modelResults = runModelOverEpochs(bestParams, XTrain, XValidate, maxEpochs)
  
  epochSearchResults = []
  for train_auc, test_auc, time, epoch in modelResults:
    epochSearchResults.append({
        'train_auc': train_auc,
        'test_auc': test_auc,
        'epoch': epoch,
        'time': time,
    })
    
#   print(epochSearchResults)
  return epochSearchResults

In [ ]:
startTime = time()

maxEpochs = 20
epochSearchResults = []
for XTrain, XValidate in XTrainValidateFolds:
  epochSearchResults.append(epochSearch(XTrain, XValidate, maxEpochs))
#   downloadCSVFile(epochSearchResults, 'epochSearchResults.csv')

totalTime = time() - startTime
if totalTime > 60:
    print('Total time: {:0.2f} minutes'.format(totalTime / 60))
else:
    print('Total time: {:0.2f} seconds'.format(totalTime))

In [ ]:
def findBestEpoch(epochSearchResults):
  previousAUC = -10
  threshold = 0.001 # If AUC doesn't increase by this much, it's not worth the extra time to run this epoch
  
  for result in epochSearchResults:
    testAUC = result['test_auc']
    if testAUC > previousAUC + threshold:
      previousAUC = testAUC
    else:
      epoch = result['epoch']
      return epoch - 1
    
bestEpochs = []
for index, epochSearchResult in enumerate(epochSearchResults):
  bestEpochFold = findBestEpoch(epochSearchResult)
  bestEpochs.append(bestEpochFold)
  print('The best number of Epochs for fold {} is {}'.format(index + 1, bestEpochFold))

bestEpoch = int(np.median(bestEpochs))
print('')
print('The best number of Epochs overall is {}'.format(bestEpoch))

In [ ]:
def plotEpochSearchResults(epochSearchResults):
  for epochSearchResult in epochSearchResults:
    epochSearchDF = pd.DataFrame(epochSearchResult)
    plt.plot(epochSearchDF['epoch'], epochSearchDF['train_auc'], color='blue')
    plt.scatter(epochSearchDF['epoch'], epochSearchDF['train_auc'], color='blue')
    plt.plot(epochSearchDF['epoch'], epochSearchDF['test_auc'], color='green')
    plt.scatter(epochSearchDF['epoch'], epochSearchDF['test_auc'], color='green')
    plt.title('AUC -- Epochs')
    plt.ylabel('AUC')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()

    plt.plot(epochSearchDF['epoch'], epochSearchDF['time'], color='red')
    plt.scatter(epochSearchDF['epoch'], epochSearchDF['time'], color='red')
    plt.title('Time -- Epochs')
    plt.ylabel('Time')
    plt.xlabel('Epoch')
    plt.legend(['train'], loc='best')
    plt.show()

  
plotEpochSearchResults(epochSearchResults)

# Sampling Amount of Data

In [ ]:
dataFractions = np.arange(0.2, 1.2, 0.2)
numRatings = len(ratingsTrainDF)
dataSizes = [int(fraction * numRatings) for fraction in dataFractions]
dataSizes

In [ ]:
def sizeSamplingSearch(dataSizes):
  sizeSamplingResults = []

  for numRatings in dataSizes:
    print('Running for size: {}'.format(numRatings))
    ratingsTrainDFSampled = ratingsTrainDF.sample(n=numRatings)
    XTrain = ratingsDFToUserMovieMatrix(ratingsTrainDFSampled, userIdToIndex, movieIdToIndex)
    train_auc, test_auc, totalTime = runModel(bestParams, XTrain, XTest, bestEpoch)
    result = {
        'numRatings': numRatings,
        'train_auc': train_auc,
        'test_auc': test_auc,
        'time': totalTime,
    }
    sizeSamplingResults.append(result)

  return sizeSamplingResults

In [ ]:
sizeSamplingResults = sizeSamplingSearch(dataSizes)


In [ ]:
def plotSizeSamplingResults(sizeSamplingResults):
  sizeSamplingDF = pd.DataFrame(sizeSamplingResults)
  
  plt.scatter(sizeSamplingDF['numRatings'], sizeSamplingDF['train_auc'], color='blue')
  plt.plot(sizeSamplingDF['numRatings'], sizeSamplingDF['train_auc'], color='blue')
  plt.scatter(sizeSamplingDF['numRatings'], sizeSamplingDF['test_auc'], color='green')
  plt.plot(sizeSamplingDF['numRatings'], sizeSamplingDF['test_auc'], color='green')
  plt.title('AUC -- Number of Ratings')
  plt.ylabel('AUC')
  plt.xlabel('Number of Ratings')
  plt.legend(['train', 'test'], loc='best')
  plt.show()
  
  plt.scatter(sizeSamplingDF['numRatings'], sizeSamplingDF['time'], color='red')
  plt.plot(sizeSamplingDF['numRatings'], sizeSamplingDF['time'], color='red')
  plt.title('Time -- Number of Ratings')
  plt.ylabel('Time')
  plt.xlabel('Number of Ratings')
  plt.legend(['train', 'test'], loc='best')
  plt.show()
  
  
plotSizeSamplingResults(sizeSamplingResults)

# Sample By Number of Users

In [ ]:
def sortUsersByDensity(ratingsDF):
  counts = ratingsDF['userId'].value_counts()
  return np.asarray(counts.keys())

In [ ]:
numUsersTotal = len(userIds)
userIdsByDensity = sortUsersByDensity(ratingsDF)
userFractions = np.arange(0.2, 1.2, 0.2)
userSizes = [int(fraction * numUsersTotal) for fraction in userFractions]
userSizes

In [ ]:
def userSamplingSearch(userSizes):
  userSamplingResultsDense = []

  for numUsers in userSizes:
    print('Running for the most dense users of size: {}'.format(numUsers))
    sampledUserIds = userIdsByDensity[:numUsers]
    userIdToIndexSampled = {}
    for index, userId in enumerate(sampledUserIds):
      userIdToIndexSampled[userId] = index
      
    ratingsTrainDFSampled = ratingsTrainDF[ratingsTrainDF['userId'].isin(sampledUserIds)]
    ratingsTestDFSampled = ratingsTestDF[ratingsTestDF['userId'].isin(sampledUserIds)]
    
    XTrain = ratingsDFToUserMovieMatrix(ratingsTrainDFSampled, userIdToIndexSampled, movieIdToIndex)
    XTest = ratingsDFToUserMovieMatrix(ratingsTestDFSampled, userIdToIndexSampled, movieIdToIndex)
    
    train_auc, test_auc, totalTime = runModel(bestParams, XTrain, XTest, bestEpoch)
    result = {
        'numUsers': numUsers,
        'train_auc': train_auc,
        'test_auc': test_auc,
        'time': totalTime,
    }
    userSamplingResultsDense.append(result)
    
    
  userSamplingResultsSparse = []
  
  for numUsers in userSizes:
    print('Running for the least dense users of size: {}'.format(numUsers))
    sampledUserIds = userIdsByDensity[(numUsersTotal - numUsers):]
    userIdToIndexSampled = {}
    for index, userId in enumerate(sampledUserIds):
      userIdToIndexSampled[userId] = index
      
    ratingsTrainDFSampled = ratingsTrainDF[ratingsTrainDF['userId'].isin(sampledUserIds)]
    ratingsTestDFSampled = ratingsTestDF[ratingsTestDF['userId'].isin(sampledUserIds)]

    
    XTrain = ratingsDFToUserMovieMatrix(ratingsTrainDFSampled, userIdToIndexSampled, movieIdToIndex)
    XTest = ratingsDFToUserMovieMatrix(ratingsTestDFSampled, userIdToIndexSampled, movieIdToIndex)
    
    train_auc, test_auc, totalTime = runModel(bestParams, XTrain, XTest, bestEpoch)
    result = {
        'numUsers': numUsers,
        'train_auc': train_auc,
        'test_auc': test_auc,
        'time': totalTime,
    }
    userSamplingResultsSparse.append(result)

  return userSamplingResultsDense, userSamplingResultsSparse

In [ ]:
userSamplingResultsDense, userSamplingResultsSparse = userSamplingSearch(userSizes)
# userSamplingResults

In [ ]:
def plotUserSamplingResults(userSamplingResultsDense, userSamplingResultsSparse):
  sizeSamplingDFDense = pd.DataFrame(userSamplingResultsDense)
  sizeSamplingDFSparse = pd.DataFrame(userSamplingResultsSparse)
  
  plt.scatter(sizeSamplingDFDense['numUsers'], sizeSamplingDFDense['train_auc'], color='blue')
  plt.scatter(sizeSamplingDFSparse['numUsers'], sizeSamplingDFSparse['train_auc'], color='#4f8ff7')
  plt.scatter(sizeSamplingDFDense['numUsers'], sizeSamplingDFDense['test_auc'], color='green')
  plt.scatter(sizeSamplingDFSparse['numUsers'], sizeSamplingDFSparse['test_auc'], color='#57f954')
  plt.plot(sizeSamplingDFDense['numUsers'], sizeSamplingDFDense['train_auc'], color='blue')
  plt.plot(sizeSamplingDFSparse['numUsers'], sizeSamplingDFSparse['train_auc'], color='#4f8ff7')
  plt.plot(sizeSamplingDFDense['numUsers'], sizeSamplingDFDense['test_auc'], color='green')
  plt.plot(sizeSamplingDFSparse['numUsers'], sizeSamplingDFSparse['test_auc'], color='#57f954')
  plt.title('AUC -- Number of Ratings')
  plt.ylabel('AUC')
  plt.xlabel('Number of Users')
  plt.legend(['train dense', 'train sparse', 'test dense', 'test sparse'], loc='best')
  plt.show()
  
  plt.scatter(sizeSamplingDFDense['numUsers'], sizeSamplingDFDense['time'], color='blue')
  plt.scatter(sizeSamplingDFSparse['numUsers'], sizeSamplingDFSparse['time'], color='#4f8ff7')
  plt.plot(sizeSamplingDFDense['numUsers'], sizeSamplingDFDense['time'], color='blue')
  plt.plot(sizeSamplingDFSparse['numUsers'], sizeSamplingDFSparse['time'], color='#4f8ff7')
  plt.title('Time -- Number of Ratings')
  plt.ylabel('Time')
  plt.xlabel('Number of Users')
  plt.legend( ['time'], loc='best')
  plt.show()
 
  
plotUserSamplingResults(userSamplingResultsDense, userSamplingResultsSparse)

# Sample By Number Of Movies

In [ ]:
def sortMoviesByDensity(ratingsDF):
  counts = ratingsDF['movieId'].value_counts()
  return np.asarray(counts.keys())

In [ ]:
numMoviesTotal = len(movieIds)
movieIdsByDensity = sortMoviesByDensity(ratingsDF)
movieFractions = np.arange(0.2, 1.2, 0.2)
movieSizes = [int(fraction * numMoviesTotal) for fraction in movieFractions]
movieSizes

In [ ]:
def movieSamplingSearch(movieSizes):
  movieSamplingResultsDense = []

  for numMovies in movieSizes:
    print('Running for the most dense movies of size: {}'.format(numMovies))
    sampledMovieIds = movieIdsByDensity[:numMovies]
    movieIdToIndexSampled = {}
    for index, movieId in enumerate(sampledMovieIds):
      movieIdToIndexSampled[movieId] = index
      
    ratingsTrainDFSampled = ratingsTrainDF[ratingsTrainDF['movieId'].isin(sampledMovieIds)]
    ratingsTestDFSampled = ratingsTestDF[ratingsTestDF['movieId'].isin(sampledMovieIds)]
    
    XTrain = ratingsDFToUserMovieMatrix(ratingsTrainDFSampled, userIdToIndex, movieIdToIndexSampled)
    XTest = ratingsDFToUserMovieMatrix(ratingsTestDFSampled, userIdToIndex, movieIdToIndexSampled)
    
    train_auc, test_auc, totalTime = runModel(bestParams, XTrain, XTest, bestEpoch)
    result = {
        'numMovies': numMovies,
        'train_auc': train_auc,
        'test_auc': test_auc,
        'time': totalTime,
    }
    movieSamplingResultsDense.append(result)
    
    
  movieSamplingResultsSparse = []
  
  for numMovies in movieSizes:
    print('Running for the least dense movies of size: {}'.format(numMovies))
    sampledMovieIds = movieIdsByDensity[(numMoviesTotal - numMovies):]
    movieIdToIndexSampled = {}
    for index, movieId in enumerate(sampledMovieIds):
      movieIdToIndexSampled[movieId] = index
      
    ratingsTrainDFSampled = ratingsTrainDF[ratingsTrainDF['movieId'].isin(sampledMovieIds)]
    ratingsTestDFSampled = ratingsTestDF[ratingsTestDF['movieId'].isin(sampledMovieIds)]

    
    XTrain = ratingsDFToUserMovieMatrix(ratingsTrainDFSampled, userIdToIndex, movieIdToIndexSampled)
    XTest = ratingsDFToUserMovieMatrix(ratingsTestDFSampled, userIdToIndex, movieIdToIndexSampled)
    
    train_auc, test_auc, totalTime = runModel(bestParams, XTrain, XTest, bestEpoch)
    result = {
        'numMovies': numMovies,
        'train_auc': train_auc,
        'test_auc': test_auc,
        'time': totalTime,
    }
    movieSamplingResultsSparse.append(result)

  return movieSamplingResultsDense, movieSamplingResultsSparse

In [ ]:
movieSamplingResultsDense, movieSamplingResultsSparse = movieSamplingSearch(movieSizes)
# userSamplingResults

In [ ]:
def plotMovieSamplingResults(movieSamplingResultsDense, movieSamplingResultsSparse):
  sizeSamplingDFDense = pd.DataFrame(movieSamplingResultsDense)
  sizeSamplingDFSparse = pd.DataFrame(movieSamplingResultsSparse)
  
  plt.scatter(sizeSamplingDFDense['numMovies'], sizeSamplingDFDense['train_auc'], color='blue')
  plt.scatter(sizeSamplingDFSparse['numMovies'], sizeSamplingDFSparse['train_auc'], color='#4f8ff7')
  plt.scatter(sizeSamplingDFDense['numMovies'], sizeSamplingDFDense['test_auc'], color='green')
  plt.scatter(sizeSamplingDFSparse['numMovies'], sizeSamplingDFSparse['test_auc'], color='#57f954')
  plt.plot(sizeSamplingDFDense['numMovies'], sizeSamplingDFDense['train_auc'], color='blue')
  plt.plot(sizeSamplingDFSparse['numMovies'], sizeSamplingDFSparse['train_auc'], color='#4f8ff7')
  plt.plot(sizeSamplingDFDense['numMovies'], sizeSamplingDFDense['test_auc'], color='green')
  plt.plot(sizeSamplingDFSparse['numMovies'], sizeSamplingDFSparse['test_auc'], color='#57f954')
  plt.title('AUC -- Number of Ratings')
  plt.ylabel('AUC')
  plt.xlabel('Number of Movies')
  plt.legend(['train dense', 'train sparse', 'test dense', 'test sparse'], loc='best')
  plt.show()
  
  plt.scatter(sizeSamplingDFDense['numMovies'], sizeSamplingDFDense['time'], color='blue')
  plt.scatter(sizeSamplingDFSparse['numMovies'], sizeSamplingDFSparse['time'], color='#4f8ff7')
  plt.plot(sizeSamplingDFDense['numMovies'], sizeSamplingDFDense['time'], color='blue')
  plt.plot(sizeSamplingDFSparse['numMovies'], sizeSamplingDFSparse['time'], color='#4f8ff7')
  plt.title('Time -- Number of Ratings')
  plt.ylabel('Time')
  plt.xlabel('Number of Movies')
  plt.legend( ['time'], loc='best')
  plt.show()
  
plotMovieSamplingResults(movieSamplingResultsDense, movieSamplingResultsSparse)